<a href="https://colab.research.google.com/github/BhagyaSree12/CNN_for_Breast_Cancer_Prediction/blob/main/Copy_of_Untitled31_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil
os.listdir("/content/drive/MyDrive/archive (1)/Breast Cancer Patients MRI's")

['validation', 'train']

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_SHAPE  = 224
batch_size = 32

without vgg16

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

base_model.summary()

base_model.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])


from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

# load and iterate training dataset
train_it = datagen.flow_from_directory("/content/drive/MyDrive/archive (1)/Breast Cancer Patients MRI's/train",
                                       target_size=(224, 224),
                                       color_mode='rgb',
                                       class_mode='binary',
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory("/content/drive/MyDrive/archive (1)/Breast Cancer Patients MRI's/validation",
                                      target_size=(224, 224),
                                      color_mode='rgb',
                                      class_mode='binary',
                                      batch_size=8)

h1= model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, workers=10, epochs=20)

Found 1410 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Epoch 1/20
12/12 [==============================] - 6s 247ms/step - loss: 1.4049 - binary_accuracy: 0.4896 - val_loss: 1.1717 - val_binary_accuracy: 0.5312
Epoch 2/20
12/12 [==============================] - 3s 225ms/step - loss: 1.0646 - binary_accuracy: 0.4583 - val_loss: 0.7231 - val_binary_accuracy: 0.7188
Epoch 3/20
12/12 [==============================] - 4s 298ms/step - loss: 0.7935 - binary_accuracy: 0.5938 - val_loss: 0.8304 - val_binary_accuracy: 0.5938
Epoch 4/20
12/12 [==============================] - 5s 327ms/step - loss: 0.8408 - binary_accuracy: 0.5521 - val_loss: 0.9421 - val_binary_accuracy: 0.6250
Epoch 5/20
12/12 [==============================] - 3s 223ms/step - loss: 0.6898 - binary_accuracy: 0.6771 - val_loss: 0.4280 - val_binary_accuracy: 0.8438
Epoch 6/20
12/12 [==============================] - 8s 620ms/step - loss: 0.7480 - binary_accuracy: 0.5312 - val_loss: 1.2664 - val_binary

with vgg16

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .000001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy','Precision','Recall','AUC'])

history = model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, workers=10, epochs=20)

Epoch 1/20
12/12 [==============================] - 9s 305ms/step - loss: 0.5242 - accuracy: 0.7292 - precision: 0.7576 - recall: 0.5814 - auc: 0.7878 - val_loss: 0.8416 - val_accuracy: 0.7188 - val_precision: 0.6250 - val_recall: 1.0000 - val_auc: 0.8235
Epoch 2/20
12/12 [==============================] - 5s 363ms/step - loss: 0.5462 - accuracy: 0.7188 - precision: 0.7576 - recall: 0.5682 - auc: 0.7670 - val_loss: 0.5735 - val_accuracy: 0.7812 - val_precision: 0.7083 - val_recall: 1.0000 - val_auc: 0.8196
Epoch 3/20
12/12 [==============================] - 5s 272ms/step - loss: 0.4822 - accuracy: 0.6979 - precision: 0.7931 - recall: 0.5000 - auc: 0.7833 - val_loss: 0.8776 - val_accuracy: 0.6562 - val_precision: 0.6400 - val_recall: 0.8889 - val_auc: 0.7183
Epoch 4/20
12/12 [==============================] - 4s 275ms/step - loss: 0.5733 - accuracy: 0.6979 - precision: 0.7037 - recall: 0.4750 - auc: 0.7156 - val_loss: 0.6284 - val_accuracy: 0.7812 - val_precision: 0.7619 - val_recall: 0

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model


In [ ]:

import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_SHAPE  = 224
batch_size = 32

resnet50

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.000001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy','Precision','Recall','AUC'])
history = model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, workers=10, epochs=50)


Epoch 1/50
12/12 [==============================] - 8s 289ms/step - loss: 0.4418 - accuracy: 0.7500 - precision: 0.7838 - recall: 0.6444 - auc: 0.7795 - val_loss: 0.7252 - val_accuracy: 0.7500 - val_precision: 0.7083 - val_recall: 0.9444 - val_auc: 0.7679
Epoch 2/50
12/12 [==============================] - 5s 396ms/step - loss: 0.4587 - accuracy: 0.7778 - precision: 0.8108 - recall: 0.6977 - auc: 0.8169 - val_loss: 0.8040 - val_accuracy: 0.7188 - val_precision: 0.6190 - val_recall: 0.9286 - val_auc: 0.7421
Epoch 3/50
12/12 [==============================] - 4s 273ms/step - loss: 0.4373 - accuracy: 0.8021 - precision: 0.8750 - recall: 0.6512 - auc: 0.8056 - val_loss: 0.8254 - val_accuracy: 0.7188 - val_precision: 0.6538 - val_recall: 1.0000 - val_auc: 0.7765
Epoch 4/50
12/12 [==============================] - 6s 431ms/step - loss: 0.3795 - accuracy: 0.8229 - precision: 0.8718 - recall: 0.7391 - auc: 0.8667 - val_loss: 0.6708 - val_accuracy: 0.7188 - val_precision: 0.6800 - val_recall: 0

transfer siamese+resnet

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Define the base ResNet-50 network (shared by both input branches)
def create_base_network(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False  # Freeze the weights of the pre-trained ResNet layers
    return base_model

# Define the two input branches of the Siamese Network
input_shape = (224, 224, 3)  # Input shape for your images
left_input = Input(shape=input_shape)
right_input = Input(shape=input_shape)

# Create two separate base networks (shared weights) for both inputs
base_network = create_base_network(input_shape)
left_output = base_network(left_input)
right_output = base_network(right_input)

# Define the distance metric (Euclidean distance) between the outputs of the base networks
distance = Lambda(lambda x: tf.keras.backend.abs(x[0] - x[1]))([left_output, right_output])

# Create the final Siamese Network model
siamese_model = Model(inputs=[left_input, right_input], outputs=distance)

# Unfreeze the base ResNet model for fine-tuning
base_network.trainable = True

# Compile the Siamese Network with a very low learning rate and binary cross-entropy loss
siamese_model.compile(optimizer=Adam(lr=0.000001),
                      loss='binary_crossentropy',
                      metrics=['accuracy','Precision','Recall','AUC'])

# Print the model summary
siamese_model.summary()


Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_33 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_34 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 resnet50 (Functional)       (None, 7, 7, 2048)           2358771   ['input_33[0][0]',            
                                                          2          'input_34[0][0]']            
                                                                                                  
 lambda_7 (Lambda)           (None, 7, 7, 2048)           0         ['resnet50[0][0]',      

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

# Load a pre-trained ResNet-50 model without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define the two input branches of the Siamese Network
input_shape = (224, 224, 3)
left_input = Input(shape=input_shape)
right_input = Input(shape=input_shape)

# Apply the same base ResNet model for both inputs
left_output = base_model(left_input)
right_output = base_model(right_input)

# Define the distance metric (Euclidean distance) between the outputs of the base networks
distance = Lambda(lambda x: tf.keras.backend.abs(x[0] - x[1]))([left_output, right_output])

# Create the Siamese Network model
siamese_model = Model(inputs=[left_input, right_input], outputs=distance)

# Unfreeze the base ResNet model for fine-tuning
base_model.trainable = True

# Compile the Siamese Network with a very low learning rate and binary cross-entropy loss
siamese_model.compile(optimizer=RMSprop(learning_rate=0.000001),
                      loss='binary_crossentropy',
                      metrics=['accuracy','Precision','Recall','AUC'])

# Print the model summary
siamese_model.summary()

# Training the Siamese Network using your data generators
# history = .fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, workers=10, epochs=20)

history = model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, workers=10, epochs=20)


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_43 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_44 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 resnet50 (Functional)       (None, 7, 7, 2048)           2358771   ['input_43[0][0]',            
                                                          2          'input_44[0][0]']            
                                                                                                  
 lambda_10 (Lambda)          (None, 7, 7, 2048)           0         ['resnet50[0][0]',     

crnn

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.layers import TimeDistributed

# Define CRNN model
def create_CRNN_model(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))

    # Reshape for LSTM layers
    model.add(Reshape((-1, 64)))

    # Recurrent layers
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.5))

    # TimeDistributed layer for sequence output
    model.add(TimeDistributed(Dense(num_classes, activation='softmax')))

    return model

# Example usage
input_shape = (224, 224, 3)  # Adjust input shape based on your MRI image dimensions
num_classes = 2  # Assuming binary classification (e.g., disease progression or not)

# Create CRNN model
crnn_model = create_CRNN_model(input_shape, num_classes)

# Compile the model
crnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy','Precision','Recall','AUC','F1'])

# Print model summary
crnn_model.summary()


Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_122 (Conv2D)         (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 112, 112, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_123 (Conv2D)         (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 56, 56, 64)        0         
 ng2D)                                                           
                                                                 
 reshape_8 (Reshape)         (None, 3136, 64)          0         
                                                                 
 bidirectional_8 (Bidirecti  (None, 3136, 256)       

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import Adam

# Define CRNN model
def create_CRNN_model(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))

    # Reshape for LSTM layers
    model.add(Reshape((-1, 64)))

    # Recurrent layers
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.5))

    # TimeDistributed layer for sequence output
    model.add(TimeDistributed(Dense(num_classes, activation='softmax')))

    return model

# Example usage
input_shape = (224, 224, 3)  # Adjust input shape based on your MRI image dimensions
num_classes = 2  # Assuming binary classification (e.g., disease progression or not)

# Create CRNN model
crnn_model = create_CRNN_model(input_shape, num_classes)

# Compile the model
crnn_model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy','Precision','Recall','AUC','F1'])

# Print model summary
crnn_model.summary()

# Train the model
epochs = 30  # Set the number of epochs
# Assuming you have train_data and validation_data for training and validation sets
history = model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, workers=10, epochs=20)

# # Evaluate the model on test data
# test_loss, test_accuracy = crnn_model.evaluate(test_data)
# print("Test Loss:", test_loss)
# print("Test Accuracy:", test_accuracy)


Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_124 (Conv2D)         (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 112, 112, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_125 (Conv2D)         (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_29 (MaxPooli  (None, 56, 56, 64)        0         
 ng2D)                                                           
                                                                 
 reshape_9 (Reshape)         (None, 3136, 64)          0         
                                                                 
 bidirectional_9 (Bidirecti  (None, 3136, 256)       

other code for vgg reset for precision,f1

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import os
from IPython.display import Image
import tensorflow.keras.backend as K
# import split-folders
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
def F1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='Accuracy'),
      tf.keras.metrics.Precision(name='Precision'),
      tf.keras.metrics.Recall(name='Recall'),
      tf.keras.metrics.AUC(name='AUC'),
      F1,
]

def plot_history(hist, name):
  fig, axs = plt.subplots(1, 3, figsize=(15, 5))
  axs[0].plot(hist.history['Accuracy'])
  axs[0].plot(hist.history['val_Accuracy'])
  axs[0].set_title(f'{name} Accuracy')
  axs[0].set_ylabel('Accuracy')
  axs[0].set_xlabel('Epoch')
  axs[0].legend(['train', 'val', 'F1', 'Recall'], loc='upper left')

  axs[1].plot(hist.history['loss'])
  axs[1].plot(hist.history['val_loss'])
  axs[1].set_title(f'{name} Loss')
  axs[1].set_ylabel('Loss')
  axs[1].set_xlabel('Epoch')
  axs[1].legend(['train', 'val'], loc='upper left')

  axs[2].plot(hist.history['Precision'])
  axs[2].plot(hist.history['val_Precision'])
  axs[2].set_title(f'{name} Precision')
  axs[2].set_ylabel('Precision')
  axs[2].set_xlabel('Epoch')
  axs[2].legend(['train', 'val'], loc='upper left')

  plt.show()

In [ ]:
VGG16_base = tf.keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights="imagenet")

for layer in VGG16_base.layers:
    layer.trainable = False

VGG16=tf.keras.Sequential()
VGG16.add(VGG16_base)
VGG16.add(tf.keras.layers.Dropout(0.5))
VGG16.add(tf.keras.layers.Flatten())
VGG16.add(tf.keras.layers.BatchNormalization())
VGG16.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
VGG16.add(tf.keras.layers.BatchNormalization())
VGG16.add(tf.keras.layers.Activation('relu'))
VGG16.add(tf.keras.layers.Dropout(0.5))
VGG16.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
VGG16.add(tf.keras.layers.BatchNormalization())
VGG16.add(tf.keras.layers.Activation('relu'))
VGG16.add(tf.keras.layers.Dropout(0.5))
VGG16.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
VGG16.add(tf.keras.layers.BatchNormalization())
VGG16.add(tf.keras.layers.Activation('relu'))
VGG16.add(tf.keras.layers.Dense(1,activation='sigmoid'))

VGG16.summary()

VGG16.compile(optimizer='rmsprop',
              loss=tf.keras.losses.binary_crossentropy, metrics=METRICS)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 dropout_2 (Dropout)         (None, 7, 7, 512)         0         
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 batch_normalization (Batch  (None, 25088)             100352    
 Normalization)                                                  
                                                                 
 dense_3 (Dense)             (None, 32)                802848    
                                                                 
 batch_normalization_1 (Bat  (None, 32)                128       
 chNormalization)                                     

In [ ]:
lrd = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',patience = 3,verbose = 1,factor = 0.50, min_lr = 1e-7)

mcp = tf.keras.callbacks.ModelCheckpoint('VGG16.h5', save_best_only=True, mode='auto', monitor='val_accuracy')

es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=3)

VGG16_history = VGG16.fit(train_it,validation_data=valid_it, epochs = 36, callbacks=[lrd,mcp,es], shuffle=True)

Epoch 1/36
177/177 [==============================] - ETA: 0s - loss: 0.6964 - Accuracy: 0.5546 - Precision: 0.5486 - Recall: 0.5800 - AUC: 0.5638 - F1: 0.5450

177/177 [==============================] - 30s 149ms/step - loss: 0.6964 - Accuracy: 0.5546 - Precision: 0.5486 - Recall: 0.5800 - AUC: 0.5638 - F1: 0.5450 - val_loss: 0.6820 - val_Accuracy: 0.5250 - val_Precision: 0.5161 - val_Recall: 0.8000 - val_AUC: 0.6097 - val_F1: 0.5969 - lr: 0.0010
Epoch 2/36
177/177 [==============================] - ETA: 0s - loss: 0.6707 - Accuracy: 0.5851 - Precision: 0.5857 - Recall: 0.5614 - AUC: 0.6248 - F1: 0.5549

177/177 [==============================] - 26s 148ms/step - loss: 0.6707 - Accuracy: 0.5851 - Precision: 0.5857 - Recall: 0.5614 - AUC: 0.6248 - F1: 0.5549 - val_loss: 0.6855 - val_Accuracy: 0.5875 - val_Precision: 0.5556 - val_Recall: 0.8750 - val_AUC: 0.6191 - val_F1: 0.6668 - lr: 0.0010
Epoch 3/36
177/177 [==============================] - ETA: 0s - loss: 0.6253 - Accuracy: 0.6411 - Precision: 0.6402 - Recall: 0.6329 - AUC: 0.7020 - F1: 0.6214

177/177 [==============================] - 26s 145ms/step - loss: 0.6253 - Accuracy: 0.6411 - Precision: 0.6402 - Recall: 0.6329 - AUC: 0.7020 - F1: 0.6214 - val_loss: 0.7024 - val_Accuracy: 0.5750 - val_Precision: 0.5484 - val_Recall: 0.8500 - val_AUC: 0.6522 - val_F1: 0.6608 - lr: 0.0010
Epoch 4/36
177/177 [==============================] - ETA: 0s - loss: 0.6177 - Accuracy: 0.6496 - Precision: 0.6439 - Recall: 0.6586 - AUC: 0.7123 - F1: 0.6355
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.


177/177 [==============================] - 27s 150ms/step - loss: 0.6177 - Accuracy: 0.6496 - Precision: 0.6439 - Recall: 0.6586 - AUC: 0.7123 - F1: 0.6355 - val_loss: 0.8285 - val_Accuracy: 0.5250 - val_Precision: 0.5143 - val_Recall: 0.9000 - val_AUC: 0.5325 - val_F1: 0.6296 - lr: 0.0010
Epoch 4: early stopping


In [ ]:
VGG16_scores = VGG16.evaluate(train_it, verbose=1)

177/177 [==============================] - 23s 129ms/step - loss: 0.4101 - Accuracy: 0.8411 - Precision: 0.8510 - Recall: 0.8243 - AUC: 0.9210 - F1: 0.8120


In [ ]:
ResNet50_base = tf.keras.applications.ResNet50(input_shape=(224,224,3),include_top=False,weights="imagenet")

for layer in ResNet50_base.layers:
    layer.trainable = False

ResNet50=tf.keras.Sequential()
ResNet50.add(ResNet50_base)
ResNet50.add(tf.keras.layers.Dropout(0.5))
ResNet50.add(tf.keras.layers.Flatten())
ResNet50.add(tf.keras.layers.BatchNormalization())
ResNet50.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
ResNet50.add(tf.keras.layers.BatchNormalization())
ResNet50.add(tf.keras.layers.Activation('relu'))
ResNet50.add(tf.keras.layers.Dropout(0.5))
ResNet50.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
ResNet50.add(tf.keras.layers.BatchNormalization())
ResNet50.add(tf.keras.layers.Activation('relu'))
ResNet50.add(tf.keras.layers.Dropout(0.5))
ResNet50.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
ResNet50.add(tf.keras.layers.BatchNormalization())
ResNet50.add(tf.keras.layers.Activation('relu'))
ResNet50.add(tf.keras.layers.Dense(1,activation='sigmoid'))

ResNet50.summary()

ResNet50.compile(optimizer='rmsprop',
              loss=tf.keras.losses.binary_crossentropy, metrics=METRICS)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 dropout_5 (Dropout)         (None, 7, 7, 2048)        0         
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 batch_normalization_4 (Bat  (None, 100352)            401408    
 chNormalization)                                                
                                                                 
 dense_7 (Dense)             (None, 32)                3211296   
                                                                 
 batch_normalization_5 (Bat  (None, 32)                128       
 chNormalization)                                     

In [ ]:
lrd = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',patience = 3,verbose = 1,factor = 0.50, min_lr = 1e-7)

mcp = tf.keras.callbacks.ModelCheckpoint('ResNet50.h5', save_best_only=True, mode='auto', monitor='val_accuracy')

es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=3)

ResNet50_history = ResNet50.fit(train_it,validation_data=valid_it,epochs = 36,verbose = 1, callbacks=[lrd, mcp, es], shuffle=True)


Epoch 1/36
177/177 [==============================] - ETA: 0s - loss: 0.7084 - Accuracy: 0.6982 - Precision: 0.6971 - Recall: 0.6936 - AUC: 0.7800 - F1: 0.5332

177/177 [==============================] - 34s 153ms/step - loss: 0.7084 - Accuracy: 0.6982 - Precision: 0.6971 - Recall: 0.6936 - AUC: 0.7800 - F1: 0.5332 - val_loss: 0.7892 - val_Accuracy: 0.4750 - val_Precision: 0.4762 - val_Recall: 0.5000 - val_AUC: 0.4466 - val_F1: 0.4531 - lr: 0.0010
Epoch 2/36
177/177 [==============================] - ETA: 0s - loss: 0.6617 - Accuracy: 0.6021 - Precision: 0.6000 - Recall: 0.5957 - AUC: 0.6506 - F1: 0.5721

177/177 [==============================] - 25s 140ms/step - loss: 0.6617 - Accuracy: 0.6021 - Precision: 0.6000 - Recall: 0.5957 - AUC: 0.6506 - F1: 0.5721 - val_loss: 0.7084 - val_Accuracy: 0.6125 - val_Precision: 0.5692 - val_Recall: 0.9250 - val_AUC: 0.5959 - val_F1: 0.6939 - lr: 0.0010
Epoch 3/36
177/177 [==============================] - ETA: 0s - loss: 0.6177 - Accuracy: 0.6525 - Precision: 0.6393 - Recall: 0.6886 - AUC: 0.7142 - F1: 0.6391

177/177 [==============================] - 26s 146ms/step - loss: 0.6177 - Accuracy: 0.6525 - Precision: 0.6393 - Recall: 0.6886 - AUC: 0.7142 - F1: 0.6391 - val_loss: 0.8226 - val_Accuracy: 0.5500 - val_Precision: 0.5294 - val_Recall: 0.9000 - val_AUC: 0.4316 - val_F1: 0.6466 - lr: 0.0010
Epoch 4/36
177/177 [==============================] - ETA: 0s - loss: 0.5851 - Accuracy: 0.6950 - Precision: 0.6795 - Recall: 0.7300 - AUC: 0.7574 - F1: 0.6831

177/177 [==============================] - 25s 143ms/step - loss: 0.5851 - Accuracy: 0.6950 - Precision: 0.6795 - Recall: 0.7300 - AUC: 0.7574 - F1: 0.6831 - val_loss: 0.8945 - val_Accuracy: 0.4875 - val_Precision: 0.4925 - val_Recall: 0.8250 - val_AUC: 0.4016 - val_F1: 0.6058 - lr: 0.0010
Epoch 5/36
177/177 [==============================] - ETA: 0s - loss: 0.5430 - Accuracy: 0.7248 - Precision: 0.7069 - Recall: 0.7614 - AUC: 0.7995 - F1: 0.7154
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.


177/177 [==============================] - 26s 146ms/step - loss: 0.5430 - Accuracy: 0.7248 - Precision: 0.7069 - Recall: 0.7614 - AUC: 0.7995 - F1: 0.7154 - val_loss: 0.9271 - val_Accuracy: 0.4625 - val_Precision: 0.4769 - val_Recall: 0.7750 - val_AUC: 0.4038 - val_F1: 0.5706 - lr: 0.0010
Epoch 5: early stopping


In [ ]:
ResNet50_scores = ResNet50.evaluate(train_it, verbose=1)

177/177 [==============================] - 24s 138ms/step - loss: 0.3411 - Accuracy: 0.8858 - Precision: 0.8579 - Recall: 0.9229 - AUC: 0.9516 - F1: 0.8786


In [ ]:
eval = [ VGG16_scores, ResNet50_scores]
eval

[[0.4100976288318634,
  0.8411347270011902,
  0.8510324358940125,
  0.8242856860160828,
  0.9209617376327515,
  0.8120183348655701],
 [0.3411182165145874,
  0.8858156204223633,
  0.8579017519950867,
  0.9228571653366089,
  0.9515714645385742,
  0.8786447048187256]]

In [ ]:
indices = ['VGG16', 'ResNet']
scores = pd.DataFrame({'Accuracy': [item[1] for item in eval],'Precision': [item[2] for item in eval], 'Recall': [item[3] for item in eval], 'AUC': [item[4] for item in eval], 'F1': [item[5] for item in eval]}, index=indices)
scores


,Accuracy,Precision,Recall,AUC,F1
VGG16,0.841135,0.851032,0.824286,0.920962,0.812018
ResNet,0.885816,0.857902,0.922857,0.951571,0.878645


In [ ]:
# from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

# # Define ReduceLROnPlateau, ModelCheckpoint, and EarlyStopping callbacks
# lrd = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.50, min_lr=1e-7)
# mcp = ModelCheckpoint('VGG16.h5', save_best_only=True, mode='auto', monitor='val_accuracy')
# es = EarlyStopping(verbose=1, patience=3)

# # Compile the Siamese Network with a very low learning rate and binary cross-entropy loss
# siamese_model.compile(optimizer=Adam(lr=0.000001),
#                       loss='binary_crossentropy',
#                       metrics=['accuracy'])

# # Training the model with callbacks
# VGG16_history = model.fit(train_it, validation_data=valid_it, epochs=36, callbacks=[lrd, mcp, es], shuffle=True)

lrd = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',patience = 3,verbose = 1,factor = 0.50, min_lr = 1e-7)

mcp = tf.keras.callbacks.ModelCheckpoint('siamese.h5', save_best_only=True, mode='auto', monitor='val_accuracy')

es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=3)

ResNet50_history = model.fit(train_it,validation_data=valid_it,epochs = 36,verbose = 1, callbacks=[lrd, mcp, es], shuffle=True)


Epoch 1/36
177/177 [==============================] - ETA: 0s - loss: 0.0274 - binary_accuracy: 0.9922

177/177 [==============================] - 32s 179ms/step - loss: 0.0274 - binary_accuracy: 0.9922 - val_loss: 1.7347 - val_binary_accuracy: 0.5625 - lr: 5.0000e-07
Epoch 2/36
177/177 [==============================] - ETA: 0s - loss: 0.0214 - binary_accuracy: 0.9950

177/177 [==============================] - 31s 177ms/step - loss: 0.0214 - binary_accuracy: 0.9950 - val_loss: 1.7943 - val_binary_accuracy: 0.5125 - lr: 5.0000e-07
Epoch 3/36
177/177 [==============================] - ETA: 0s - loss: 0.0311 - binary_accuracy: 0.9915

177/177 [==============================] - 32s 179ms/step - loss: 0.0311 - binary_accuracy: 0.9915 - val_loss: 1.7976 - val_binary_accuracy: 0.5625 - lr: 5.0000e-07
Epoch 4/36
177/177 [==============================] - ETA: 0s - loss: 0.0276 - binary_accuracy: 0.9915
Epoch 4: ReduceLROnPlateau reducing learning rate to 2.499999993688107e-07.


177/177 [==============================] - 31s 176ms/step - loss: 0.0276 - binary_accuracy: 0.9915 - val_loss: 1.8512 - val_binary_accuracy: 0.4375 - lr: 5.0000e-07
Epoch 4: early stopping


In [ ]:
snn_scores = model.evaluate(train_it, verbose=1)

177/177 [==============================] - 25s 139ms/step - loss: 0.0496 - binary_accuracy: 0.9872


In [ ]:
eval = [ snn_scores]
eval

[[0.04956305772066116, 0.9872340559959412]]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

class MetricsCallback(tf.keras.callbacks.Callback):
    def _init_(self, validation_data):
        super(MetricsCallback, self)._init_()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_predictions = self.model.predict(self.validation_data[0])
        val_labels = self.validation_data[1]
        val_predictions_binary = (val_predictions > 0.5).astype(int)

        val_precision = precision_score(val_labels, val_predictions_binary)
        val_recall = recall_score(val_labels, val_predictions_binary)
        val_f1 = f1_score(val_labels, val_predictions_binary)

        print(f'val_precision: {val_precision:.4f} - val_recall: {val_recall:.4f} - val_f1: {val_f1:.4f}')
        return

In [ ]:
import tensorflow as tf

# Define the ResNet-50 base
ResNet50_base = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Freeze the layers in the ResNet-50 base
for layer in ResNet50_base.layers:
    layer.trainable = False

# Create the CRNN model
CRNN = tf.keras.Sequential()

# Add the ResNet-50 base
CRNN.add(ResNet50_base)

# Add a 2D TimeDistributed layer to apply recurrent layers over the spatial dimension
CRNN.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))

# Add a SimpleRNN or LSTM layer
CRNN.add(tf.keras.layers.SimpleRNN(32, return_sequences=True))  # You can use SimpleRNN or LSTM

# Add a Dense layer after the recurrent layer
CRNN.add(tf.keras.layers.Dense(32, kernel_initializer='he_uniform'))
CRNN.add(tf.keras.layers.BatchNormalization())
CRNN.add(tf.keras.layers.Activation('relu'))
CRNN.add(tf.keras.layers.Dropout(0.5))

# Add the output layer
CRNN.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Print the model summary
CRNN.summary()

# Compile the CRNN model
CRNN.compile(optimizer='rmsprop',
            loss=tf.keras.losses.binary_crossentropy,
            metrics=['accuracy'])

# You can use your desired metrics or define METRICS as needed


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 time_distributed_3 (TimeDi  (None, 7, 14336)          0         
 stributed)                                                      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 7, 32)             459808    
                                                                 
 dense_13 (Dense)            (None, 7, 32)             1056      
                                                                 
 batch_normalization_9 (Bat  (None, 7, 32)             128       
 chNormalization)                                                
                                                                 
 activation_7 (Activation)   (None, 7, 32)            

In [ ]:
import tensorflow as tf

# Assuming you've already defined your CRNN model
# (Refer to the previous code example for creating the CRNN model)

lrd = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.50, min_lr=1e-7)

mcp = tf.keras.callbacks.ModelCheckpoint('CRNN.h5', save_best_only=True, mode='auto', monitor='val_accuracy')

es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=3)

CRNN_history = CRNN.fit(train_it, validation_data=valid_it, epochs=36, verbose=1, callbacks=[lrd, mcp, es], shuffle=True)


Epoch 1/36


ValueError: ignored

In [ ]:
InceptionV3_base = tf.keras.applications.InceptionV3(input_shape=(224,224,3),include_top=False,weights="imagenet")

for layer in InceptionV3_base.layers:
    layer.trainable = False

InceptionV3=tf.keras.Sequential()
InceptionV3.add(InceptionV3_base)
InceptionV3.add(tf.keras.layers.Dropout(0.5))
InceptionV3.add(tf.keras.layers.Flatten())
InceptionV3.add(tf.keras.layers.BatchNormalization())
InceptionV3.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
InceptionV3.add(tf.keras.layers.BatchNormalization())
InceptionV3.add(tf.keras.layers.Activation('relu'))
InceptionV3.add(tf.keras.layers.Dropout(0.5))
InceptionV3.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
InceptionV3.add(tf.keras.layers.BatchNormalization())
InceptionV3.add(tf.keras.layers.Activation('relu'))
InceptionV3.add(tf.keras.layers.Dropout(0.5))
InceptionV3.add(tf.keras.layers.Dense(32,kernel_initializer='he_uniform'))
InceptionV3.add(tf.keras.layers.BatchNormalization())
InceptionV3.add(tf.keras.layers.Activation('relu'))
InceptionV3.add(tf.keras.layers.Dense(1,activation='sigmoid'))

InceptionV3.summary()

InceptionV3.compile(optimizer='rmsprop',
              loss=tf.keras.losses.binary_crossentropy, metrics=METRICS)

87910968/87910968 [==============================] - 1s 0us/step
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 dropout_10 (Dropout)        (None, 5, 5, 2048)        0         
                                                                 
 flatten_4 (Flatten)         (None, 51200)             0         
                                                                 
 batch_normalization_104 (B  (None, 51200)             204800    
 atchNormalization)                                              
                                                                 
 dense_15 (Dense)            (None, 32)                1638432   
                                                                 
 batch_normalization_105 (B  (None, 32)                

In [ ]:
lrd = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',patience = 3,verbose = 1,factor = 0.50, min_lr = 1e-7)

mcp = tf.keras.callbacks.ModelCheckpoint('InceptionV3.h5', save_best_only=True, mode='auto', monitor='val_accuracy')

es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=3)

InceptionV3_history = InceptionV3.fit(train_it,validation_data=valid_it,epochs = 36,verbose = 1, callbacks=[lrd, mcp, es], shuffle=True)


Epoch 1/36
177/177 [==============================] - ETA: 0s - loss: 0.7566 - Accuracy: 0.6993 - Precision: 0.6749 - Recall: 0.7607 - AUC: 0.7856 - F1: 0.5237

177/177 [==============================] - 41s 158ms/step - loss: 0.7566 - Accuracy: 0.6993 - Precision: 0.6749 - Recall: 0.7607 - AUC: 0.7856 - F1: 0.5237 - val_loss: 0.6435 - val_Accuracy: 0.6000 - val_Precision: 0.5870 - val_Recall: 0.6750 - val_AUC: 0.6791 - val_F1: 0.6271 - lr: 0.0010
Epoch 2/36
177/177 [==============================] - ETA: 0s - loss: 0.7172 - Accuracy: 0.5326 - Precision: 0.5303 - Recall: 0.5129 - AUC: 0.5349 - F1: 0.4927

177/177 [==============================] - 28s 156ms/step - loss: 0.7172 - Accuracy: 0.5326 - Precision: 0.5303 - Recall: 0.5129 - AUC: 0.5349 - F1: 0.4927 - val_loss: 0.5927 - val_Accuracy: 0.7375 - val_Precision: 0.6863 - val_Recall: 0.8750 - val_AUC: 0.7781 - val_F1: 0.7480 - lr: 0.0010
Epoch 3/36
177/177 [==============================] - ETA: 0s - loss: 0.7094 - Accuracy: 0.5319 - Precision: 0.5296 - Recall: 0.5114 - AUC: 0.5508 - F1: 0.4948

177/177 [==============================] - 28s 157ms/step - loss: 0.7094 - Accuracy: 0.5319 - Precision: 0.5296 - Recall: 0.5114 - AUC: 0.5508 - F1: 0.4948 - val_loss: 0.7059 - val_Accuracy: 0.5625 - val_Precision: 0.5610 - val_Recall: 0.5750 - val_AUC: 0.5853 - val_F1: 0.5407 - lr: 0.0010
Epoch 4/36
177/177 [==============================] - ETA: 0s - loss: 0.7032 - Accuracy: 0.5383 - Precision: 0.5371 - Recall: 0.5071 - AUC: 0.5492 - F1: 0.4950

177/177 [==============================] - 29s 166ms/step - loss: 0.7032 - Accuracy: 0.5383 - Precision: 0.5371 - Recall: 0.5071 - AUC: 0.5492 - F1: 0.4950 - val_loss: 0.6495 - val_Accuracy: 0.7250 - val_Precision: 0.7368 - val_Recall: 0.7000 - val_AUC: 0.7741 - val_F1: 0.6813 - lr: 0.0010
Epoch 5/36
177/177 [==============================] - ETA: 0s - loss: 0.6856 - Accuracy: 0.5844 - Precision: 0.5833 - Recall: 0.5700 - AUC: 0.5969 - F1: 0.5510
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.


177/177 [==============================] - 26s 145ms/step - loss: 0.6856 - Accuracy: 0.5844 - Precision: 0.5833 - Recall: 0.5700 - AUC: 0.5969 - F1: 0.5510 - val_loss: 0.6166 - val_Accuracy: 0.7125 - val_Precision: 0.6809 - val_Recall: 0.8000 - val_AUC: 0.7384 - val_F1: 0.7172 - lr: 0.0010
Epoch 5: early stopping


In [ ]:
import tensorflow as tf

# Define the base ResNet-50 architecture
ResNet50_base = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Create a Siamese network
input_shape = (224, 224, 3)
input_a = tf.keras.layers.Input(shape=input_shape)
input_b = tf.keras.layers.Input(shape=input_shape)

# Branch 1
branch1 = tf.keras.Sequential([
    ResNet50_base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, kernel_initializer='he_uniform'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu')
])

# Branch 2
branch2 = tf.keras.Sequential([
    ResNet50_base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, kernel_initializer='he_uniform'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu')
])

# Merge the branches by computing L1 distance
distance = tf.keras.layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([branch1(input_a), branch2(input_b)])

# Make a binary prediction
output = tf.keras.layers.Dense(1, activation='sigmoid')(distance)

# Create the Siamese model
SiameseModel = tf.keras.models.Model(inputs=[input_a, input_b], outputs=output)

# Print the model summary
SiameseModel.summary()

# Compile the Siamese model
SiameseModel.compile(optimizer='rmsprop',
                     loss=tf.keras.losses.binary_crossentropy,
                     metrics=['accuracy','AUC','Precision','Recall'])

# Assuming you have your data prepared for Siamese training
# x_train_pair1 and x_train_pair2 should contain pairs of images
# y_train is the binary similarity/dissimilarity label for each pair
model.fit(train_it,validation_data=valid_it, epochs=36, verbose=1)


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_25 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_26 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 sequential_15 (Sequential)  (None, 32)                   2366160   ['input_25[0][0]']            
                                                          0                                       
                                                                                                  
 sequential_16 (Sequential)  (None, 32)                   2366160   ['input_26[0][0]']      

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(64,activation='relu'))
model.add(GaussianNoise(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_104 (Conv2D)         (None, 222, 222, 32)      896       
                                                                 
 conv2d_105 (Conv2D)         (None, 220, 220, 32)      9248      
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 110, 110, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_106 (Conv2D)         (None, 108, 108, 64)      18496     
                                                                 
 conv2d_107 (Conv2D)         (None, 106, 106, 64)      36928     
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 53, 53, 64)        0         
 ng2D)                                               

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','AUC','Precision','Recall'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
es=EarlyStopping(patience=3,monitor='val_loss')
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history = model.fit(
    train_it,
    epochs=10,
    validation_data=valid_it,
    steps_per_epoch= 50,
    callbacks=checkpoint
    )

Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 0.8143 - accuracy: 0.5050 - auc: 0.5249 - precision: 0.4706 - recall: 0.4255
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_model.h5
50/50 [==============================] - 15s 150ms/step - loss: 0.8143 - accuracy: 0.5050 - auc: 0.5249 - precision: 0.4706 - recall: 0.4255 - val_loss: 0.6917 - val_accuracy: 0.5000 - val_auc: 0.7000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.7230 - accuracy: 0.4975 - auc: 0.4461 - precision: 0.5182 - recall: 0.5455
Epoch 2: val_accuracy did not improve from 0.50000
50/50 [==============================] - 9s 189ms/step - loss: 0.7230 - accuracy: 0.4975 - auc: 0.4461 - precision: 0.5182 - recall: 0.5455 - val_loss: 0.6928 - val_accuracy: 0.5000 - val_auc: 0.5713 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 0.6887 - accuracy: 0.5400 - auc: 0.5540 - precision: 0.5527 - recall: 0.7136
Epoch 3: val_accuracy did not improve from 0.50000
50/50 [==============================] - 7s 147ms/step - loss: 0.6887 - accuracy: 0.5400 - auc: 0.5540 - precision: 0.5527 - recall: 0.7136 - val_loss: 0.6835 - val_accuracy: 0.5000 - val_auc: 0.6666 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 4/10
50/50 [==============================] - ETA: 0s - loss: 0.6940 - accuracy: 0.5254 - auc: 0.5475 - precisio